# Paso 1: Cargar los archivos Excel
Utilizaremos `pandas` para cargar ambos archivos Excel.

In [6]:
import pandas as pd

# Cargar los archivos Excel
padron_df = pd.read_excel('PADRON ACTIVIDAD 21-05-24.xlsx')
rendicion_df = pd.read_excel('RENDICON VENTAS COD608 P05-24 (1).xlsx', skiprows=7)

# Mostrar una vista preliminar de los datos
print(padron_df.head())
print(rendicion_df.head())

   Credencial  Apellido           Nombre Destino Delegacion
0       23411   NANCUAN    VICTOR ADRIAN      DN         DA
1       24078   MAIDANA          ROBERTO      34         34
2       25186     TULLO  LILIANA ANTONIA      DN         DA
3       27323  CEVALLOS  MARIA MAGDALENA       2          2
4       27534  GONZALEZ   SANDRA FABIANA      13         13
  Unid     Cred Apellido y Nombres  Cuota de Mes Fecha de Operación   Factura  \
0  U19  36051.0   MATIAS FERNANDEZ        1272.0         2024-03-18   50184.0   
1  U19  37213.0    JESICA VILLAGRA        1344.0         2024-03-20   50185.0   
2  U19  47512.0    ELIANA AYZAGUER        1208.0         2024-03-20   50186.0   
3  U19  37038.0     VALERIA BOGADO        1320.0         2024-03-20   50187.0   
4  U19  42360.0  DARIO HUENTEMILLA        3272.0         2024-03-20   50188.0   

   Monto Total   Remito  Cuota Computada  Cuota Nº Detalle de la compra  \
0       1272.0      NaN              1.0       1.0       QUESO PATEGRAS   
1  

c:\Python312\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


# Paso 2: Normalizar los nombres de las columnas y unir "Apellido" y "Nombre"
Vamos a unir las columnas "Apellido" y "Nombre" en `padron_df` para que coincidan con el formato del segundo archivo, y luego renombramos la columna combinada.

In [2]:
# Unir "Apellido" y "Nombre" en una sola columna
padron_df['Apellido y Nombres'] = padron_df['Apellido'] + ' ' + padron_df['Nombre']

# Renombrar columnas en ambos DataFrames para asegurar consistencia
padron_df = padron_df.rename(columns={'Credencial': 'Cred'})
rendicion_df = rendicion_df.rename(columns={'Cred': 'Cred', 'Apellido y Nombres': 'Apellido y Nombres'})

# Eliminar las columnas originales "Apellido" y "Nombre" ya que ya están combinadas
padron_df = padron_df.drop(columns=['Apellido', 'Nombre'])

# Mostrar una vista preliminar para confirmar
print(padron_df.head())
print(rendicion_df.head())

    Cred Destino Delegacion        Apellido y Nombres
0  23411      DN         DA     NANCUAN VICTOR ADRIAN
1  24078      34         34           MAIDANA ROBERTO
2  25186      DN         DA     TULLO LILIANA ANTONIA
3  27323       2          2  CEVALLOS MARIA MAGDALENA
4  27534      13         13   GONZALEZ SANDRA FABIANA
  Unid     Cred Apellido y Nombres  Cuota de Mes Fecha de Operación   Factura  \
0  U19  36051.0   MATIAS FERNANDEZ        1272.0         2024-03-18   50184.0   
1  U19  37213.0    JESICA VILLAGRA        1344.0         2024-03-20   50185.0   
2  U19  47512.0    ELIANA AYZAGUER        1208.0         2024-03-20   50186.0   
3  U19  37038.0     VALERIA BOGADO        1320.0         2024-03-20   50187.0   
4  U19  42360.0  DARIO HUENTEMILLA        3272.0         2024-03-20   50188.0   

   Monto Total   Remito  Cuota Computada  Cuota Nº Detalle de la compra  \
0       1272.0      NaN              1.0       1.0       QUESO PATEGRAS   
1       1344.0      NaN              1.

# Paso 3: Comparar los datos entre los DataFrames
Recorreremos el segundo Excel (`rendicion_df`) y verificaremos si la `Cred` y `Apellido y Nombres` coinciden con los datos del primer Excel (`padron_df`).


In [3]:
# Crear una nueva columna en rendicion_df para indicar si la fila es correcta o tiene errores
rendicion_df['Validación'] = 'Error'

# Función para validar si la credencial y el nombre completo existen en padron_df
def validar_fila(row):
    # Buscar si la credencial y el nombre coinciden en padron_df
    # padron_fila = padron_df[(padron_df['Cred'] == row['Cred']) & 
    #                         (padron_df['Apellido y Nombres'] == row['Apellido y Nombres'])]
    padron_fila = padron_df[(padron_df['Cred'] == row['Cred'])] 
                            # (padron_df['Apellido y Nombres'] == row['Apellido y Nombres'])]
    # Si hay coincidencia, marcar como "Correcto"
    if not padron_fila.empty:
        return 'Correcto'
    # Si no hay coincidencia, marcar como "Error"
    return 'Error'

# Aplicar la validación a cada fila de rendicion_df
rendicion_df['Validación'] = rendicion_df.apply(validar_fila, axis=1)

# Mostrar una vista previa para verificar el resultado
print(rendicion_df[['Cred', 'Apellido y Nombres', 'Validación']].head())

      Cred Apellido y Nombres Validación
0  36051.0   MATIAS FERNANDEZ   Correcto
1  37213.0    JESICA VILLAGRA   Correcto
2  47512.0    ELIANA AYZAGUER   Correcto
3  37038.0     VALERIA BOGADO   Correcto
4  42360.0  DARIO HUENTEMILLA   Correcto


# Paso 4: Guardar el archivo con la validación
Guardamos el archivo resultante con la nueva columna "Validación" que indica si cada fila es correcta o contiene errores.


In [5]:
# Definir el nombre del archivo de salida
output_path = 'RENDICON VENTAS COD608 P05-24 (CONTROLADO).xlsx'

# Guardar el resultado en un archivo Excel
rendicion_df.to_excel(output_path, index=False)

# Confirmar que el archivo fue guardado
print(f'Archivo guardado en: {output_path}')

Archivo guardado en: RENDICON VENTAS COD608 P05-24 (CONTROLADO).xlsx
